<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparação-do-Ambiente" data-toc-modified-id="Preparação-do-Ambiente-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparação do Ambiente</a></span></li><li><span><a href="#Conexão-ao-Banco-de-Dados" data-toc-modified-id="Conexão-ao-Banco-de-Dados-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Conexão ao Banco de Dados</a></span></li><li><span><a href="#Objetos-do-Banco-de-Dados" data-toc-modified-id="Objetos-do-Banco-de-Dados-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Objetos do Banco de Dados</a></span><ul class="toc-item"><li><span><a href="#Insumos" data-toc-modified-id="Insumos-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Insumos</a></span></li></ul></li></ul></div>

Uma das mais poderosas funcionalidades proporcionadas pela linguagem Python é justamente a facilidade em se trabalhar com dados. Em situações genéricas, o pool de conexões a bancos de dados também abre um leque gigantesco de possibilidades, permitindo assim com que usuários gerenciem, consultem e automatizem tarefas envolvendo persistências de dados utilizando diferentes SGBDs.

Neste cenário, considerando o desenvolvimento prévio composto pelo notebook de documentação da API `nba_api`, bem como a construção das classes responsáveis por consumir tal pacote python e retornar dados estatísticos da NBA dentro das propostas definidas, este notebook pretende alocar as etapas introdutórias para utilização de um banco de dados para armazenamento dos dados extraídos da API. Essa atuação, de alguma forma, será extremamente importante para propor um entendimento mais claro sobre como a solução como um todo pode ser integrada a outros sistemas e plataformas.

Para isso, será utilizado o **PostgreSQL** como SGBD principal e a biblioteca [`psycopg2`](https://pypi.org/project/psycopg2/) que permite a conexão entre Python e o SGBD. O comando para instalação da biblioteca diretamente no jupyter notebook é:

        !pip install psycopg2
        
<div align="center">
    <img src="https://cpl.thalesgroup.com/sites/default/files/content/paragraphs/intro/2020-03/postgresql-logo.png" height=500 width=300 alt="postgresql logo">
</div>

# Preparação do Ambiente

Após a [instalação](https://www.youtube.com/watch?v=wuAdsns3qMA&list=PLucm8g_ezqNoAkYKXN_zWupyH6hQCAwxY&index=5) do [PostgreSQL](https://www.postgresql.org/download/) no sistema operacional utilizado e a configuração do ambiente a partir da [criação de novos usuários](https://www.youtube.com/watch?v=XqkGdFO-ENc&list=PLucm8g_ezqNoAkYKXN_zWupyH6hQCAwxY&index=8) e definição de [políticas de senha](https://www.youtube.com/watch?v=HbAJQ1WuGRE&list=PLucm8g_ezqNoAkYKXN_zWupyH6hQCAwxY&index=7), é preciso criar um banco de dados. Essa criação pode ser realizada a partir do utilitário `psql` diretamente no prompt de comando ou então utilizando a interface gráfica proposta pelo [`pgAdmin4`](https://www.pgadmin.org/download/).

Para criação de um novo banco de dados no postgres, é preciso realizar o login utilizando um usuário com permissões de criação de bancos de dados e, no ambiente postgres, executar o comando `CREATE DATABASE <nome_database>;`. A imagem abaixo mostra os detalhes sobre esse procedimento a partir da criação do banco de dados _nbaflow_ utilizando o usuário _paninit_.

<div align="center">
    <br><img src="https://i.imgur.com/bCdtHLu.png" height=800 width=800 alt="postgresql logo">
</div>

# Conexão ao Banco de Dados

Uma vez criado o banco de dados, é possível realizar a conexão de um usuário a partir da função `connect()` do pacote `psycopg2`. A função `connect()` cria uma nova sessão do banco de dados e retorna uma instância da classe "conexão". Com esse novo objeto, é possível criar cursores para executar comandos SQL. Para executar a função `connect()` é preciso especificar, em um primeiro momento, parâmetros relacionados ao banco de dados do PostgreSQL como uma string de conexão.

In [1]:
# Adicionando diretório do ambiente virtual como uma variável de ambiente para importação dos pacotes
import sys
sys.path.append('/home/paninit/python-venvs/nbaflow-venv/lib/python3.8/site-packages')

# Biblioteca postgres
import psycopg2

# Importando bibliotecas
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
# Lendo variáveis de ambiente
load_dotenv(find_dotenv())

# Definindo variáveis de diretório
HOST = '127.0.0.1'
DB_USER = 'paninit'
DB_NAME = 'nbaflow'

# Criando objeto de conexão
conn = psycopg2.connect(host=HOST,
                        database=DB_NAME,
                        user=DB_USER,
                        password=os.getenv('DB_PASSWORD'))

# Criando cursor e testando conexão
cur = conn.cursor()
print(f'Conexão realizada com sucesso no PostgreSQL versão:\n')
cur.execute('SELECT version()')
print(cur.fetchone()[0])
cur.close()

Conexão realizada com sucesso no PostgreSQL versão:

PostgreSQL 11.12 (Ubuntu 11.12-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit


No bloco acima, foi definido um exemplo de conexão com o banco de dados PostgreSQL utilizando credenciais passadas pelo próprio usuário. Como uma das informações requisitadas para a conexão é a senha do usuário, é preciso pensar em alternativas para mascarar essa informação e deixá-la inacessível para outras aplicações. Nesse primeiro momento, informações sensíveis (como a senha, por exemplo) podem ser armazenadas em um arquivo `.env` localizado no direorio do projeto, sendo sua leitura gerenciada pelas funções `find_dotenv()` e `load_dotenv()`. Dessa forma, é possível simular a inserção de variáveis como se fossem variáveis de ambiente do sistema operacional, permitindo assim sua posterior coleta utilizando a função `os.getenv()`.

Eventualmente, uma outra alternativa visa a criação de um [arquivo de conexão](https://www.postgresqltutorial.com/postgresql-python/connect/) para armazenar as informações sensíveis de conexão.

# Objetos do Banco de Dados

Após a apresentação do método `connect()` da biblioteca `psycopg2` para realizar a conexão a um banco de dados PostgreSQL, é possível encapsular códigos capazes de manusear os objetos dentro do banco de dados com base em elementos externos. Em outras palavras, é possível investigar a possibilidade de executar um comando `CREATE TABLE` a partir dos nomes e tipos primitivos de uma base de dados originalmente dada em um objeto DataFrame do pandas.

Nesta sessão, investigações serão realizadas nessa linha tendo, como premissa, a base de dados de partidas (gamelog) extraída no notebook de apresentação da API `nba_api`.

## Insumos

In [3]:
# Importando pandas e lendo base de dados
import pandas as pd
pd.set_option('display.max_columns', 500)

gamelog = pd.read_csv('../data/players_gamelog.csv')
print(f'Dimensões da base de dados lida: {gamelog.shape}')
gamelog.head()

Dimensões da base de dados lida: (159414, 29)


,season_id,player_id,player_name,game_id,game_date,matchup,wl,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,tov,pf,pts,plus_minus,video_available,season_type
0,42019,203500,Steven Adams,41900177,2020-09-02,OKC @ HOU,L,34,4,6,0.667,0,0,0.0,2,3,0.667,4,5,9.0,0,2,1,1,0,10,0,1,Playoffs
1,42019,203500,Steven Adams,41900176,2020-08-31,OKC vs. HOU,W,31,3,9,0.333,0,0,0.0,0,0,0.000,9,5,14.0,1,1,0,4,1,6,-4,1,Playoffs
2,42019,203500,Steven Adams,41900175,2020-08-29,OKC @ HOU,L,25,6,8,0.750,0,0,0.0,0,2,0.000,8,6,14.0,0,0,0,0,1,12,-15,1,Playoffs
3,42019,203500,Steven Adams,41900174,2020-08-24,OKC vs. HOU,W,26,4,5,0.800,0,0,0.0,4,6,0.667,4,4,8.0,1,0,0,3,1,12,-7,1,Playoffs
4,42019,203500,Steven Adams,41900173,2020-08-22,OKC vs. HOU,W,35,3,7,0.429,0,0,0.0,0,3,0.000,2,11,13.0,2,0,1,3,4,6,5,1,Playoffs


Acima, é apresentada a base de dados de partidas de cada um dos jogadores da NBA em todas as temporadas de atuação dos mesmos. Para maiores detalhes sobre como esse dado foi extraído, é possível checar o material disponível no [link](https://github.com/ThiagoPanini/nbaflow/blob/main/notebooks/NBAflow-doc-api.ipynb).

Bom, após a leitura e visualização da base, é possível executar o atributo `dtypes` do pandas para visualizar os nomes das colunas e seus respectivos tipos primitivos.

In [4]:
# Tipagem da base
gamelog.dtypes

season_id            int64
player_id            int64
player_name         object
game_id              int64
game_date           object
matchup             object
wl                  object
min                  int64
fgm                  int64
fga                  int64
fg_pct             float64
fg3m                 int64
fg3a                 int64
fg3_pct            float64
ftm                  int64
fta                  int64
ft_pct             float64
oreb                 int64
dreb                 int64
reb                float64
ast                  int64
stl                  int64
blk                  int64
tov                  int64
pf                   int64
pts                  int64
plus_minus           int64
video_available      int64
season_type         object
dtype: object

A partir desse momento, considerando os conhecimentos básicos e os blocos fundamentais da linguagem SQL, seria possível utilizar as informações das colunas e dos tipos primitivos fornecidos pelo atributo `dtype` para definir, individualmente, o schema a ser considerado no comando `CREATE TABLE` (e posteriormente no comando `INSERT INTO` para inserção dos dados na tabela). Neste cenário, o usuário teria total liberdade para definir o schema por conta própria, gerenciando contraints e demais fatores customizados que poderiam fazer parte da tabela.

Entretanto, fatalmente usuários desenvolveram técnicas mais fáceis de transcrever DataFrames em tabelas SQL

_PRÓXIMOS PASSOS_

    - Criação de tabela com o layout da base de gamelog extraída
    - Tentativa de criação dinâmica de tabela utilizando o schema extraído do próprio DataFrame
        * Encapsulamento de script
        * Alteração de parâmetros (nome de colunas e tipos primitivos)
        * Execução do script de criação, se aplicável
    - Proposta de ingestão de dados a partir de um DataFrame (INSERT INTO)